<a href="https://colab.research.google.com/github/rudycav/RedditScraper/blob/main/Project/r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [445]:
!pip install asyncpraw
!pip install praw
!pip install nltk
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [446]:
import asyncpraw
import praw
from praw.models import MoreComments
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re
import string
from string import punctuation
from emoji import demojize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [447]:
reddit = praw.Reddit(
    client_id="BE3T0CMpNrtqLT97yL6XiA",
    client_secret = "gJnvUXTK0gCnj3UMGSUtw0JW35tN1A",
    password = "Hannah00!!@",
    user_agent = "spidey by u/rudyboycav",
    username = "rudyboycav",
)

In [448]:
def reddit_scraper(reddit, subreddit_page, limit):
  df = pd.DataFrame()
  subreddit = reddit.subreddit(subreddit_page)
  hot_topic = subreddit.hot(limit=limit)

  for x in hot_topic: 
    c = x.comments.list()
    for y in c:
      if isinstance(y, MoreComments):
        continue
      df = df.append({#'subreddit': subreddit_page,
                      #'id': x.id,
                      #'title': x.title,
                      #'author': x.author,
                      'comments': y.body},
                      ignore_index=True) 

  return df

In [449]:
wall = reddit_scraper(reddit, 'wallstreetbets', 2)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [450]:
wall.head()

,comments
0,Lol how the fuck is 2022 almost over
1,CNBC: Futures little changed. \n\nWSB: REEEEEE...
2,I should have stayed cash.
3,"I wish this sub existed back in 2008, I‘d real..."
4,Bul gang: 😰\n\nTheta gang: also 😰\n\n🌽 gang: 💀...


In [451]:
def clean_text(df):
  clean_text = " ".join(str(df).split())
  clean_text = " ".join([w for w in df.split() if w.isalpha()])

  return clean_text

gg = wall.comments.map(clean_text)

In [468]:
for x in gg:
  print(x)

Lol how the fuck is almost over
Futures little
I should have stayed
I wish this sub existed back in really like to read some of the conversations back then everything dumped
Bul Theta also Ber
every sunday the last idk week we will focus more on our day job and closing deals because this is sustainable consistent income and trading is your side hustle and monday friday FUCK THIS FUCKING MARKET LETS GOOOOOO FUCK its already work tomorrow for sure this
corn better be by open
Everyone is expecting market to sell off
not an I just have to drink every day to avoid the shakes
Tried to get tattooed on my shaft but only able to get on there
Real remember when you could get fat as FUCK in GTA
Starting a new job in the city where my gf lives and she want me to move in with She wants me to get my own Sad
Has corn ever been profitable since El Salvador adopted
All ye who survive this market will be millionaires next bull market
FUTURES FALL FUTURES
I know what going to do tomorrow but I can guaran

In [453]:
def word_tokenizer(df):
  tokens = word_tokenize(str(df))

  return tokens

ggg =word_tokenizer(gg)

In [466]:
print(len(ggg))

84


In [454]:
stopwords_ = set(stopwords.words("english"))

def stopwords(df):
  clean_tokens = [t for t in df if not t in stopwords_]
  clean_text = " ".join(clean_tokens)

  return clean_text

gggg = stopwords(ggg)

In [ ]:
for x in gggg:
  print(x)

In [455]:
non_letter = "".join([x for x in str(cleaned_text) if x not in string.punctuation])
non_letter

'None'

In [456]:
def remove_punctuation(df):

  #non_letter = re.sub(r"https?://\S+", "", str(df))
  non_letter = "".join([x for x in str(df) if x not in string.punctuation]) #removes punctuation
  non_letter = ''.join([i for i in str(df) if not i.isdigit()]) #removes numbers
  #non_letter = demojize(str(df))
  
  return non_letter

cleaned_text = remove_punctuation(cleaned_text)

In [457]:
cleaned_text

'None'

In [458]:
for x in cleaned_text:
  if x is 'SPY':
    print(x)

In [459]:
for x in cleaned_text:
  if x.isupper():
    print(x)

N


In [460]:
def tokenize(df):
  for words in wall:
    tokenize = word_tokenize(words)
  return tokenize

tokenized_words = tokenize(wall)

In [461]:
stop_words = set(stopwords.words("english"))

AttributeError: ignored

In [ ]:
def stopwords(df):
  #stop_words = set(stopwords.words("english"))
  filtered_words = [word for word in df if word.casefold() not in stop_words]
  return filtered_words

filtered_stopwords = stopwords(tokenized_words)
filtered_stopwords

In [ ]:
def stemming(df):
  stemmed_list = []
  snow_stemmer = SnowballStemmer(language='english')
  
  for words in filtered_stopwords:
    stemmer = snow_stemmer.stem(words)
    stemmed_list.append(stemmer)
  return stemmed_list

stemmed_words = stemming(filtered_stopwords)
stemmed_words

In [ ]:
subreddit = await reddit.subreddit("wallstreetbets")
async for x in subreddit.hot(limit=3):
    print('THREAD: {}'.format(x.title))
    print('USER: {}'.format(x.author))

    print(x.comments)
    

